## HOMER on tile accessibility

### Top and bottom 1/4th

__Combine the tiles per replicate__

In [ ]:
# Combine the the replicates per cell line
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data


    # MM001 alone
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-B_MM001_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM001_COMBINED_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt ; done

    # 'MM029_OLS-A','MM029_OLS-B'
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM029_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM029_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM029_COMBINED_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt ; done

    # 'MM057_OLS-A','MM057_OLS-B'
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM057_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM057_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM057_COMBINED_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt ; done

    # MM074 alone
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM074_rep2_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM074_COMBINED_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt ; done

    # 'MM087_OLS-A_round_2','MM087_OLS-B'
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM087_comb_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM087_rep3_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM087_COMBINED_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt ; done

    # 3 repl MM099
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM099_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt Tiles_OLS-A_MM099_rep2_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM099_rep1_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM099_COMBINED_5-5BC_ATAC-CNnorm_FILT-topbottom4th_${type}-tiles.txt ; done

__Format files for Homer__

In [ ]:
# Convert the tiles to bed format
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data

for file in Tiles_M*.txt; do \
 cat $file | tr "@@" "\t"  | tr ":" "\t" | tr "-" "\t" | awk '{print $1 "\t" $2 "\t" $3 "\t" $1 ":" $2 "-" $3 "@@" $4 }' > ${file%%.*}.bed ; done

# Remove entries that are mutated and the shuffled tiles
for file in *.bed; do \
 cat $file | grep -v "mut" | grep -v "shuffled" > ${file%%.*}.noMut.bed ; done

# Convert bed to fasta
module load BEDTools
for file in *.noMut.bed; do \
bedtools getfasta \
-fi /staging/leuven/stg_00002/lcb/resources/human/hg19/Homo_sapiens_assembly19_sorted.fa \
-bed $file -name -fo ${file%%.*}.noMut.fa ; done

rm *.bed

__Homer on combined CNnorm tiles__

In [ ]:
# All accessible vs all inaccessible
    # using fastas, and no de novo motif enrichment

FILTERING='ATAC-CNnorm_FILT-topbottom4th'
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
                findMotifs.pl \
                Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-accessible-tiles.noMut.fa \
                fasta \
                Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_all-accessible-tiles_vs_all-inaccessible-tiles/ \
                -mask \
                -nomotif \
                -len 6,8,10,11,12,17,18 \
                -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inaccessible-tiles.noMut.fa \
        ; done 

# All inaccessible vs all accessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inaccessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inaccessible-tiles_vs_all-accessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-accessible-tiles.noMut.fa \
        ; done 


# Prol accessible vs prol inaccessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-accessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-accessible-tiles_vs_all-prol-inaccessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inaccessible-tiles.noMut.fa \
        ; done

# Prol inaccessible vs prol accessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inaccessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inaccessible-tiles_vs_all-prol-accessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-accessible-tiles.noMut.fa \
        ; done

# Inv accessible vs inv inaccessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-accessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-accessible-tiles_vs_all-inv-inaccessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inaccessible-tiles.noMut.fa \
        ; done

# Inv inaccessible vs inv accessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inaccessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inaccessible-tiles_vs_all-inv-accessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-accessible-tiles.noMut.fa \
        ; done


### Top 1/4th and bottom 3/4th

__Combine the tiles per replicate__

In [ ]:
# Combine the the replicates per cell line
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data

    # MM001 alone
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-B_MM001_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM001_COMBINED_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt ; done

    # 'MM029_OLS-A','MM029_OLS-B'
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM029_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt Tiles_OLS-B_MM029_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM029_COMBINED_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt ; done

    # 'MM057_OLS-A','MM057_OLS-B'
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM057_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt Tiles_OLS-B_MM057_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM057_COMBINED_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt ; done

    # MM074 alone
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM074_rep2_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM074_COMBINED_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt ; done

    # 'MM087_OLS-A_round_2','MM087_OLS-B'
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM087_comb_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt Tiles_OLS-B_MM087_rep3_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM087_COMBINED_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt ; done

    # 3 repl MM099
for type in 'all-accessible' 'all-inaccessible' 'prol-accessible' 'prol-inaccessible' 'inv-accessible' 'inv-inaccessible'; do \
cat Tiles_OLS-A_MM099_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt Tiles_OLS-A_MM099_rep2_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt Tiles_OLS-B_MM099_rep1_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM099_COMBINED_5-5BC_ATAC-CNnorm_FILT-top4thbottom3-4th_${type}-tiles.txt ; done

__Format files for Homer__

In [ ]:
# Convert the tiles to bed format
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data

for file in Tiles_M*top4thbottom3-4th*.txt; do \
 cat $file | tr "@@" "\t"  | tr ":" "\t" | tr "-" "\t" | awk '{print $1 "\t" $2 "\t" $3 "\t" $1 ":" $2 "-" $3 "@@" $4 }' > ${file%%.*}.bed ; done

# Remove entries that are mutated and the shuffled tiles
for file in *top4thbottom3-4th*.bed; do \
 cat $file | grep -v "mut" | grep -v "shuffled" > ${file%%.*}.noMut.bed ; done

# Convert bed to fasta
module load BEDTools
for file in *top4thbottom3-4th*.noMut.bed; do \
bedtools getfasta \
-fi /staging/leuven/stg_00002/lcb/resources/human/hg19/Homo_sapiens_assembly19_sorted.fa \
-bed $file -name -fo ${file%%.*}.noMut.fa ; done

rm *.bed

__Homer on combined CNnorm tiles__

In [ ]:
# All accessible vs all inaccessible
    # using fastas, and no de novo motif enrichment

FILTERING='ATAC-CNnorm_FILT-top4thbottom3-4th'
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
                findMotifs.pl \
                Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-accessible-tiles.noMut.fa \
                fasta \
                Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_all-accessible-tiles_vs_all-inaccessible-tiles/ \
                -mask \
                -nomotif \
                -len 6,8,10,11,12,17,18 \
                -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inaccessible-tiles.noMut.fa \
        ; done 

# All inaccessible vs all accessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inaccessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inaccessible-tiles_vs_all-accessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-accessible-tiles.noMut.fa \
        ; done 


# Prol accessible vs prol inaccessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-accessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-accessible-tiles_vs_all-prol-inaccessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inaccessible-tiles.noMut.fa \
        ; done

# Prol inaccessible vs prol accessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inaccessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inaccessible-tiles_vs_all-prol-accessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-accessible-tiles.noMut.fa \
        ; done

# Inv accessible vs inv inaccessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-accessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-accessible-tiles_vs_all-inv-inaccessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inaccessible-tiles.noMut.fa \
        ; done

# Inv inaccessible vs inv accessible
    # using fastas, and no de novo motif enrichment

cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER       
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inaccessible-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inaccessible-tiles_vs_all-inv-accessible-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-accessible-tiles.noMut.fa \
        ; done


## HOMER on tile activity

__Combine the tiles per replicate__

In [ ]:
# Combine the the replicates per cell line
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data


    # MM001 alone
for type in 'all-active' 'all-inactive' 'prol-active' 'prol-inactive' 'inv-active' 'inv-inactive'; do \
cat Tiles_OLS-B_MM001_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM001_COMBINED_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt ; done

    # 'MM029_OLS-A','MM029_OLS-B'
for type in 'all-active' 'all-inactive' 'prol-active' 'prol-inactive' 'inv-active' 'inv-inactive'; do \
cat Tiles_OLS-A_MM029_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM029_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM029_COMBINED_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt ; done

    # 'MM057_OLS-A','MM057_OLS-B'
for type in 'all-active' 'all-inactive' 'prol-active' 'prol-inactive' 'inv-active' 'inv-inactive'; do \
cat Tiles_OLS-A_MM057_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM057_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM057_COMBINED_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt ; done

    # MM074 alone
for type in 'all-active' 'all-inactive' 'prol-active' 'prol-inactive' 'inv-active' 'inv-inactive'; do \
cat Tiles_OLS-A_MM074_rep2_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM074_COMBINED_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt ; done

    # 'MM087_OLS-A_round_2','MM087_OLS-B'
for type in 'all-active' 'all-inactive' 'prol-active' 'prol-inactive' 'inv-active' 'inv-inactive'; do \
cat Tiles_OLS-A_MM087_comb_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM087_rep3_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM087_COMBINED_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt ; done

    # 3 repl MM099
for type in 'all-active' 'all-inactive' 'prol-active' 'prol-inactive' 'inv-active' 'inv-inactive'; do \
cat Tiles_OLS-A_MM099_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt Tiles_OLS-A_MM099_rep2_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt Tiles_OLS-B_MM099_rep1_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt | sort | uniq \
> Tiles_MM099_COMBINED_5-5BC_CHEQseq-topbottom4th_${type}-tiles.txt ; done

__Format files for Homer__

In [ ]:
# Convert the tiles to bed format
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data

for file in Tiles_M*CHEQseq*.txt; do \
 cat $file | tr "@@" "\t"  | tr ":" "\t" | tr "-" "\t" | awk '{print $1 "\t" $2 "\t" $3 "\t" $1 ":" $2 "-" $3 "@@" $4 }' > ${file%%.*}.bed ; done

# Remove entries that are mutated and the shuffled tiles
for file in Tiles_M*CHEQseq*.bed; do \
 cat $file | grep -v "mut" | grep -v "shuffled" > ${file%%.*}.noMut.bed ; done

# Convert bed to fasta
module load BEDTools
for file in *CHEQseq*.noMut.bed; do \
bedtools getfasta \
-fi /staging/leuven/stg_00002/lcb/resources/human/hg19/Homo_sapiens_assembly19_sorted.fa \
-bed $file -name -fo ${file%%.*}.noMut.fa ; done

rm *.bed

__Homer on combined CHEQseq tiles__

In [ ]:
FILTERING='CHEQseq-topbottom4th'
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/analysis/Homer/data
module load HOMER

# All active vs all inactive
    # using fastas, and no de novo motif enrichment
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
                findMotifs.pl \
                Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-active-tiles.noMut.fa \
                fasta \
                Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_all-active-tiles_vs_all-inactive-tiles/ \
                -mask \
                -nomotif \
                -len 6,8,10,11,12,17,18 \
                -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inactive-tiles.noMut.fa \
        ; done 

# All inactive vs all active
    # using fastas, and no de novo motif enrichment      
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inactive-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_all-inactive-tiles_vs_all-active-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_all-active-tiles.noMut.fa \
        ; done 


# Prol active vs prol inactive
    # using fastas, and no de novo motif enrichment      
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-active-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-active-tiles_vs_all-prol-inactive-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inactive-tiles.noMut.fa \
        ; done

# Prol inactive vs prol active
    # using fastas, and no de novo motif enrichment      
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inactive-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-inactive-tiles_vs_all-prol-active-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_prol-active-tiles.noMut.fa \
        ; done

# Inv active vs inv inactive
    # using fastas, and no de novo motif enrichment  
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-active-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-active-tiles_vs_all-inv-inactive-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inactive-tiles.noMut.fa \
        ; done

# Inv inactive vs inv active
    # using fastas, and no de novo motif enrichment
    for LINE in 'MM001' 'MM029' 'MM047' 'MM057' 'MM074' 'MM087' 'MM099' ; do \
            findMotifs.pl \
            Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inactive-tiles.noMut.fa \
            fasta \
            Homer_5-5BC_${FILTERING}_COMBINED/Homer_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-inactive-tiles_vs_all-inv-active-tiles/ \
            -mask \
            -nomotif \
            -len 6,8,10,11,12,17,18 \
            -fastaBg Tiles_${LINE}_COMBINED_5-5BC_${FILTERING}_inv-active-tiles.noMut.fa \
        ; done